<a href="https://colab.research.google.com/github/laiananardi/speech_to_text/blob/main/speechtotext_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [208]:
!pip install SpeechRecognition
!pip install pydub

In [209]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.utils import make_chunks
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Projeto Aplicado - DLE/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [210]:
def abrir_audio(file):
  
  filename_wav = f'{path}dataset_audios/{file}.wav'
  #selecionando o audio
  audio = AudioSegment.from_file(filename_wav, 'wav')

  #tamanho da divisão do audio em milisegundos
  tamanho = 30000

  #dividindo o audio 
  partes = make_chunks(audio, tamanho)
  partes_audio = []

  for i, parte in enumerate(partes):
    #enumerando arquivo particionado
    parte_name = f'{path}/audios_particoes/file{i}.wav'
    #guardando os nomes das partições em uma lista
    partes_audio.append(parte_name)
    #exportando os arquivos de audio
    parte.export(parte_name, format='wav')
  return partes_audio


In [211]:
def transcreve_audio(nome_audio):
  #selecionar audio para reconhecimento
  r = sr.Recognizer()
  with sr.AudioFile(nome_audio) as source:
    r.adjust_for_ambient_noise(source)
    audio = r.record(source)#ler audio
   

  #reconhecimento com google speech recognition
  try:
      texto = r.recognize_google(audio, language='pt-BR')
      #print(f'Google Speech Recognition: {texto}')
  except sr.UnknownValueError:
      texto = ''
      print(f'Google Speech Recognition não entendeu o audio')   
  except sr.RequestError:
      texto = ''
      print(f'Erro ao solicitar resultados do serviço Google Speech Recognition')   
  return texto


In [212]:
def transcricao():
  texto = ''
  for parte in partes_audio:
    texto = texto + ' '+ transcreve_audio(parte)
  print(texto)


In [213]:
#realiza as transcricoes e adiciona em uma lista
transcricao_python = []
for i in range(41,120):
  if i < 100:
    file = f'LapsBM_00{i}'
  else:
    file = f'LapsBM_0{i}'
  partes_audio = abrir_audio(file)
  texto = ''
  for parte in partes_audio:
    texto = f'{texto} {transcreve_audio(parte)}'
  transcricao_python.append(texto)


In [214]:
#abre os txts com as transcrições originais dos audios e coloca em uma lista

transcricao_txt = []
for i in range(41,120):
  if i < 100:
    file = f'LapsBM_00{i}'
  else:
    file = f'LapsBM_0{i}'
  
  filename_txt = f'{path}dataset_audios/{file}.txt'
  arquivo = open(filename_txt, 'r')
  texto_txt = arquivo.read()
  arquivo.close()
  transcricao_txt.append(texto_txt)


In [215]:
#coloca as transcricoes do script e as originais em um dataframe
import pandas as pd

df = pd.DataFrame((zip(transcricao_python, transcricao_txt)), columns = ['transcricao_python','transcricao_txt'])

In [216]:
#accuracia das transcrições feitas pelo script por audio
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

df['accuracy_texto'] = df['transcricao_txt'].apply(lambda x: process.extractOne(x, df['transcricao_python'].to_list(),score_cutoff=0))

In [217]:
#adiciona uma coluna apenas com o valor da acuracia de cada audio
df['accuracy'] = df['accuracy_texto'].str[-1]
df

,transcricao_python,transcricao_txt,accuracy_texto,accuracy
0,quanto ao seu próprio time o diagnóstica meno...,quanto ao seu próprio time o diagnóstico é men...,( quanto ao seu próprio time o diagnóstica men...,97
1,desculpa esse para o mal jogo que fizemos con...,não há desculpas para o mau jogo que fizemos c...,( desculpa esse para o mal jogo que fizemos co...,88
2,real média do varejo aumenta 12% no mês passado,salário real médio do varejo aumenta doze por ...,( real média do varejo aumenta 12% no mês pass...,86
3,Tokyo o dólar fechou cotado 98n,em tóquio o dólar fechou cotado a noventa e o...,"( Tokyo o dólar fechou cotado 98n, 86)",86
4,os Sobreviventes cremar os seus mortos e proc...,os sobreviventes cremaram seus mortos e procur...,( os Sobreviventes cremar os seus mortos e pro...,97
...,...,...,...,...
74,sua é um compromisso Irreversível do governo ...,o mercosul é um compromisso irreversível do go...,( sua é um compromisso Irreversível do governo...,92
75,mata da Alfândega foi 5 vezes mais cara,a mamata da alfândega foi cinco vezes mais car...,"( mata da Alfândega foi 5 vezes mais cara, 88)",88
76,secretário da cultura com apoio de intelectua...,chega a secretário da cultura com amplo apoio ...,( secretário da cultura com apoio de intelectu...,95
77,tu não pode ver orelha de eleitor que dá o ma...,petista não pode ver orelha de eleitor que dá ...,( tu não pode ver orelha de eleitor que dá o m...,94


In [218]:
#acuracia final de todos os audios que foram transcritos
accuracy = df['accuracy'].mean()
print(accuracy)
df['accuracy'].describe()


91.30379746835443


count    79.000000
mean     91.303797
std       4.133523
min      86.000000
25%      86.000000
50%      92.000000
75%      95.000000
max      99.000000
Name: accuracy, dtype: float64